In [ ]:
!gdown 1YiV9ku40vrVigPYNe41AOXZwClKbZ-Oo
!gdown 1hKTk7XlOEOoJvo65JzYXulsq02O5jSUD
!gdown 1G1wynkz7l4FlUnjSvLTOupTMom2u-sTL

Downloading...
From: https://drive.google.com/uc?id=1YiV9ku40vrVigPYNe41AOXZwClKbZ-Oo
To: /content/RuMedPrimeData.tsv
100% 7.72M/7.72M [00:00<00:00, 62.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1hKTk7XlOEOoJvo65JzYXulsq02O5jSUD
To: /content/gener_data.csv
100% 1.63M/1.63M [00:00<00:00, 173MB/s]
Downloading...
From: https://drive.google.com/uc?id=1G1wynkz7l4FlUnjSvLTOupTMom2u-sTL
To: /content/Итоговая_БАЗА_болезней.csv
100% 2.34M/2.34M [00:00<00:00, 184MB/s]


In [ ]:
import pandas as pd
import numpy as np
import json

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

SEED = 21

In [ ]:
df = pd.read_csv('RuMedPrimeData.tsv', sep='\t')
df.head()

,symptoms,anamnesis,icd10,new_patient_id,new_event_id,new_event_time
0,"Сухость кожи, мышечная слабость, нарушение мен...",Месяц назад сильный стресс и на этом фоне появ...,E01.8,qf156c36,q5fc2cb1,2027-05-19
1,"Жалобы ГБ: головная боль, головокружение на фо...","Начало острое, постепенное, после переохлажден...",J06.9,q9321cf8,qe173f20,2023-03-24
2,"головная боль, озноб, насморк, легкая заложенн...","Начало острое, *ДАТА* после переохлаждения. Пр...",J00,qcbc06c5,q653efaa,2026-12-22
3,"На изжога периодически , отрыжка воздухом, . б...",Впервые пояились вышеизложенные жалобы в течен...,K21.0,qfd53b1f,qe48681b,2026-01-15
4,"- лабильность настроения, плаксивость.","Тиреоидэктомия в*ДАТА* по поводу МУЗ, по гисто...",E89.0,q783a54b,q4109cab,2021-05-28


In [ ]:
df2 = pd.read_csv('Итоговая_БАЗА_болезней.csv')

In [ ]:
dict(df['icd10'].value_counts())

{'I11.9': 354,
 'E06.3': 210,
 'G54.4': 202,
 'M54.2': 177,
 'M54.8': 160,
 'G20': 151,
 'M54.5': 133,
 'J06.8': 132,
 'G90.9': 113,
 'G44.2': 109,
 'N41.1': 107,
 'M54.4': 106,
 'M54.6': 102,
 'M13.9': 91,
 'E66.0': 88,
 'G90.8': 83,
 'G54.2': 83,
 'I67.8': 65,
 'K21.0': 57,
 'M02.9': 56,
 'I10': 56,
 'N76.1': 55,
 'J00': 55,
 'G52.8': 55,
 'N20.0': 51,
 'D25.1': 49,
 'G98': 48,
 'E66.8': 47,
 'E11.8': 43,
 'K29.6': 43,
 'E04.1': 42,
 'G44.1': 42,
 'N40': 42,
 'N30.2': 41,
 'D50.9': 41,
 'K81.1': 41,
 'E01.8': 40,
 'J18.9': 40,
 'N86': 39,
 'M19.0': 39,
 'I20.8': 37,
 'J35.0': 37,
 'N76.0': 37,
 'E04.2': 37,
 'E05.0': 36,
 'J42': 35,
 'N80.0': 35,
 'N95.2': 34,
 'E89.0': 34,
 'J45.8': 33,
 'N83.2': 33,
 'K29.3': 31,
 'J30.4': 31,
 'K80.1': 30,
 'I83.9': 30,
 'M51.1': 28,
 'J30': 28,
 'M54.1': 28,
 'J31.2': 28,
 'L30.9': 27,
 'E22.1': 27,
 'M05.8': 27,
 'J06.9': 27,
 'M53.0': 27,
 'K81.8': 25,
 'N95.1': 25,
 'J20.9': 24,
 'M42.1': 24,
 'M19.9': 24,
 'M17.1': 23,
 'M77.3': 23,
 'N30.0':

## Данные

In [ ]:
df['icd10'] = df['icd10'].apply(lambda s: s.split('.')[0])
df = df[['symptoms', 'icd10']].reset_index(drop=True)

In [ ]:
df

,symptoms,icd10
0,"Сухость кожи, мышечная слабость, нарушение мен...",E01
1,"Жалобы ГБ: головная боль, головокружение на фо...",J06
2,"головная боль, озноб, насморк, легкая заложенн...",J00
3,"На изжога периодически , отрыжка воздухом, . б...",K21
4,"- лабильность настроения, плаксивость.",E89
...,...,...
7620,"Насморк,затек слизи по задней стенке.Высыпания...",J30
7621,"на одышку в покое, при незначительной физическ...",J45
7622,на периодическую боль в поясничной области,M54
7623,"Боли в левом плечевом суставе, усилив. в полож...",M19


In [ ]:
dict(df['icd10'].value_counts())

{'M54': 734,
 'I11': 360,
 'G54': 304,
 'E06': 220,
 'G90': 215,
 'G44': 171,
 'J06': 162,
 'G20': 151,
 'E66': 136,
 'M13': 116,
 'N76': 112,
 'N41': 108,
 'K29': 107,
 'E04': 95,
 'M19': 90,
 'G40': 85,
 'J30': 81,
 'N20': 78,
 'D25': 76,
 'K81': 75,
 'K21': 72,
 'E11': 70,
 'N30': 70,
 'I67': 67,
 'J45': 65,
 'E01': 65,
 'N95': 65,
 'E05': 58,
 'M02': 58,
 'I10': 56,
 'G52': 55,
 'J00': 55,
 'L30': 54,
 'N83': 52,
 'N80': 50,
 'M17': 50,
 'G98': 48,
 'M53': 48,
 'N97': 45,
 'J35': 44,
 'J18': 43,
 'M75': 43,
 'N40': 42,
 'D50': 41,
 'I20': 39,
 'N86': 39,
 'G25': 39,
 'K80': 38,
 'J31': 38,
 'G43': 38,
 'M51': 37,
 'H81': 37,
 'J42': 35,
 'L40': 34,
 'K58': 34,
 'E89': 34,
 'I49': 33,
 'G62': 32,
 'J84': 32,
 'L20': 32,
 'I83': 31,
 'L21': 30,
 'M05': 30,
 'J01': 30,
 'N81': 30,
 'N84': 30,
 'M77': 30,
 'N39': 29,
 'E03': 29,
 'N93': 28,
 'M15': 28,
 'E22': 28,
 'L70': 27,
 'N91': 27,
 'M42': 26,
 'N28': 25,
 'M06': 25,
 'J20': 24,
 'E28': 24,
 'J44': 23,
 'K26': 23,
 'M10': 22,
 'G

In [ ]:
parts = np.split(df.sample(frac=1, random_state=SEED), [int(0.735*len(df)), int(0.8675*len(df))])

In [ ]:
train_data = parts[0]
dev_data = parts[1]
test_data = parts[2]

In [ ]:
icd_freq = (train_data['icd10'].value_counts() > 10)
labels = [k for k, v in dict(icd_freq).items() if v]

In [ ]:
train_data = train_data[train_data['icd10'].isin(labels)]
dev_data = dev_data[dev_data['icd10'].isin(labels)]
test_data = test_data[test_data['icd10'].isin(labels)]

In [ ]:
# df_list = [train_data, dev_data, test_data]
# for i in range(0, 3):
#     print(df_list[i].shape)
#     print(df_list[i]['icd10'].value_counts())

## TFIDF и LogReg

In [ ]:
def hit_at_n(y_true, y_pred, n=3):
    assert len(y_true) == len(y_pred)
    hit_count = 0
    for l, row in zip(y_true, y_pred):
        order = (np.argsort(row)[::-1])[:n]
        hit_count += int(l in order)
    return round(hit_count / float(len(y_true)) * 100, 2)


def encode_text(tfidf, text_data, labels, l2i, mode='train'):
    if mode == 'train':
        X = tfidf.fit_transform(text_data)
    else:
        X = tfidf.transform(text_data)
    y = labels.map(l2i)
    return X, y


def logits2codes(logits, i2l, n=3):
    codes = []
    for row in logits:
        order = np.argsort(row)[::-1]
        codes.append([i2l[i] for i in order[:n]])
    return codes

In [ ]:
text_id = 'symptoms'
label_id = 'icd10' 
index_id = 'idx'

i2l = dict(enumerate(sorted(train_data[label_id].unique())))
l2i = {label: i for i, label in i2l.items()}

In [ ]:
tfidf = TfidfVectorizer(analyzer='char', ngram_range=(3, 8))
clf = LogisticRegression(penalty='l2', C=10, multi_class='ovr', n_jobs=10, verbose=1)

X, y = encode_text(tfidf, train_data[text_id], train_data[label_id], l2i)

In [ ]:
# print(X.shape)
# print(tfidf.get_feature_names_out()[::2000])

In [ ]:
clf.fit(X, y)

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:  4.2min
[Parallel(n_jobs=10)]: Done 108 out of 108 | elapsed: 13.0min finished


LogisticRegression(C=10, multi_class='ovr', n_jobs=10, verbose=1)

In [ ]:
X_val, y_val = encode_text(tfidf, dev_data[text_id], dev_data[label_id], l2i, mode='val')
y_val_pred = clf.predict_proba(X_val)

accuracy = hit_at_n(y_val, y_val_pred, n=1)
hit_3 = hit_at_n(y_val, y_val_pred, n=3)
print (f'\nscores on dev set: {accuracy} / {hit_3}')


scores on dev set: 49.04 / 74.4


In [ ]:
X_test, _ = encode_text(tfidf, test_data[text_id], test_data[label_id], l2i, mode='test')
y_test_pred = clf.predict_proba(X_test)

accuracy = hit_at_n(_, y_test_pred, n=1)
hit_3 = hit_at_n(_, y_test_pred, n=3)
print (f'\nscores on test set: {accuracy} / {hit_3}')

test_codes = logits2codes(y_test_pred, i2l)

recs = []
for true, pred in zip(test_data[label_id], test_codes):
    recs.append({label_id: true, 'prediction': pred})


scores on test set: 49.29 / 73.94


In [ ]:
out_fname = 'results.jsonl'
with open(out_fname, 'w') as fw:
    for rec in recs:
        json.dump(rec, fw, ensure_ascii=False)
        fw.write('\n')